Finding Circuits in LLM when forced to do Self Explanations

## Setup and Imports

In [ ]:

# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2


Running as a Colab notebook
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1

In [ ]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

from transformers import AutoModelForCausalLM, AutoTokenizer
import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

In [ ]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


In [ ]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

## Self-Explanations - Circuits Experimentation

In [ ]:
model = HookedTransformer.from_pretrained(
    "qwen1.5-1.8b-chat"
)
# Get the default device used
device: torch.device = utils.get_device()

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model qwen1.5-1.8b-chat into HookedTransformer


In [ ]:
# Load the pre-trained HookedTransformer model

example_prompt = "Henry found a Wolf in the forest... Who ran away? Why? -"

example = "Explanation"
print(model.to_single_token(example))
print(model.to_tokens(example_prompt))
example_answer = " Henry ran away, because he's a human. Humans are scared of Wolves"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

69769
tensor([[151645,  63363,   1730,    264,  25230,    304,    279,  13638,   1112,
          10479,  10613,   3123,     30,   8429,     30,    481]],
       device='cuda:0')
Tokenized prompt: ['<|im_end|>', 'Henry', ' found', ' a', ' Wolf', ' in', ' the', ' forest', '...', ' Who', ' ran', ' away', '?', ' Why', '?', ' -']
Tokenized answer: [' Henry', ' ran', ' away', ',', ' because', ' he', "'s", ' a', ' human', '.', ' Humans', ' are', ' scared', ' of', ' Wolves']


Performance on answer token:
Rank: 798      Logit:  8.20 Prob:  0.01% Token: | Henry|

Top 0th token. Logit: 14.95 Prob:  7.69% Token: | a|
Top 1th token. Logit: 14.92 Prob:  7.48% Token: | human|
Top 2th token. Logit: 14.83 Prob:  6.79% Token: | A|
Top 3th token. Logit: 14.49 Prob:  4.86% Token: | The|
Top 4th token. Logit: 14.03 Prob:  3.05% Token: | humans|
Top 5th token. Logit: 13.40 Prob:  1.63% Token: | the|
Top 6th token. Logit: 13.24 Prob:  1.38% Token: | Wolf|
Top 7th token. Logit: 13.24 Prob:  1.38% Token: | Dog|
Top 8th token. Logit: 13.21 Prob:  1.34% Token: | Human|
Top 9th token. Logit: 13.13 Prob:  1.25% Token: | friend|


Performance on answer token:
Rank: 16       Logit: 12.44 Prob:  0.44% Token: | ran|

Top 0th token. Logit: 17.01 Prob: 42.63% Token: | -|
Top 1th token. Logit: 15.61 Prob: 10.51% Token: | is|
Top 2th token. Logit: 15.24 Prob:  7.26% Token: | was|
Top 3th token. Logit: 14.76 Prob:  4.50% Token: |<|im_end|>|
Top 4th token. Logit: 14.36 Prob:  3.00% Token: |'s|
Top 5th token. Logit: 14.26 Prob:  2.73% Token: |\n|
Top 6th token. Logit: 14.03 Prob:  2.15% Token: | had|
Top 7th token. Logit: 13.92 Prob:  1.93% Token: | went|
Top 8th token. Logit: 13.56 Prob:  1.35% Token: | and|
Top 9th token. Logit: 13.15 Prob:  0.90% Token: | has|


...

Performance on answer token:
Rank: 0        Logit: 22.51 Prob: 94.44% Token: | of|

Top 0th token. Logit: 22.51 Prob: 94.44% Token: | of|
Top 1th token. Logit: 19.40 Prob:  4.20% Token: | by|
Top 2th token. Logit: 17.62 Prob:  0.71% Token: | and|
Top 3th token. Logit: 16.06 Prob:  0.15% Token: | away|
Top 4th token. Logit: 15.49 Prob:  0.08% Token: | to|
Top 5th token. Logit: 14.80 Prob:  0.04% Token: | easily|
Top 6th token. Logit: 14.54 Prob:  0.03% Token: | animals|
Top 7th token. Logit: 14.48 Prob:  0.03% Token: |.|
Top 8th token. Logit: 14.43 Prob:  0.03% Token: | when|
Top 9th token. Logit: 14.34 Prob:  0.03% Token: | or|


Performance on answer token:
Rank: 6        Logit: 18.04 Prob:  0.25% Token: | Wolves|

Top 0th token. Logit: 23.99 Prob: 95.47% Token: | wolves|
Top 1th token. Logit: 19.53 Prob:  1.10% Token: | big|
Top 2th token. Logit: 19.30 Prob:  0.88% Token: | wild|
Top 3th token. Logit: 18.75 Prob:  0.51% Token: | animals|
Top 4th token. Logit: 18.29 Prob:  0.32% Token: | dogs|
Top 5th token. Logit: 18.05 Prob:  0.25% Token: | the|
Top 6th token. Logit: 18.04 Prob:  0.25% Token: | Wolves|
Top 7th token. Logit: 17.69 Prob:  0.18% Token: | predators|
Top 8th token. Logit: 17.65 Prob:  0.17% Token: | large|
Top 9th token. Logit: 16.91 Prob:  0.08% Token: |狼|


Ranks of the answer tokens: [(' Henry', 798), (' ran', 16), (' away', 0), (',', 10), (' because', 0), (' he', 0), 
("'s", 29), (' a', 2), (' human', 8), ('.', 2), (' Humans', 8), (' are', 0), (' scared', 6), (' of', 0), (' 
Wolves', 6)]

In the following part: my idea was to create three prompts, GWEN will generate answers for these prompts,

I did a bit of cheating here, because I setup the temperature to 0 in order to trigger the same answers all the time, I realised it was more complicated than expected with different length explanations, and I couldn't really trigger explanations with similar words and length to make a "good" analysis of the logits after.

However: my original idea was something like

Good Explanation = "Human had to run away, because he's a human. Humans are really scared of Wolves", "Wolf ran away, because he's an animal. Animals are scared of Humans",

LLM Explanation =

In [ ]:
# Get the tokenizer from the model
tokenizer = model.tokenizer

prompts = [
    "In the forest two living creatures encountered each other, Henry and the Wolf. One of them was scared, and ran away.\n Who did run away? Why?\n",
    "In the jungle two living creatures encountered each other, Lion and Gazelle. One of them was scared and ran away.\n Who did run away? Why?\n",
    "In the rainforest two living creatures encountered each other, Jaguar and Monkey. One of creature was scared and ran away.\n Who did run away? Why?\n",
]

explanation = [
    "Human had to run away, because it was scared of the Wolf", "Wolf had to run away, because it was scared of the Human",
    "Gazelle ran away, because it was scared of the Lion", "Lion ran away, because it was scared of the Gazelle",
    "Monkey had to run away, because it was scared of the Jaguar", "Jaguar run away, because it was scared of the Monkey"
    ]

generation = []
explanations_tokens = []
generation_tokens = []


In [ ]:
# Generate text for each prompt and collect results
for prompt in prompts:
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate text
    generated_output = model.generate(
        input_ids,
        max_new_tokens=14,  # Maximum number of new tokens to generate
        top_k=25,
        top_p=0.65,
        temperature = 0.10,# Sampling probability
        do_sample=True
    )

    # Decode the generated output
    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    print(generated_text)
    answer = generated_text.split('\n')[-1].strip()

    # Append the generated answer to the list
    generation.append(answer)


  0%|          | 0/14 [00:00<?, ?it/s]

In the forest two living creatures encountered each other, Henry and the Wolf. One of them was scared, and ran away.
 Who did run away? Why?
Henry ran away because he was scared of the Wolf. The Wolf is


  0%|          | 0/14 [00:00<?, ?it/s]

In the jungle two living creatures encountered each other, Lion and Gazelle. One of them was scared and ran away.
 Who did run away? Why?
Answer: Lion. Lion was scared of Gazelle because Gazelle was


  0%|          | 0/14 [00:00<?, ?it/s]

In the rainforest two living creatures encountered each other, Jaguar and Monkey. One of creature was scared and ran away.
 Who did run away? Why?
Answer: The Jaguar ran away because it was scared of the Monkey.


In [ ]:
print(explanation)
print(generation)

['Human had to run away, because it was scared of the Wolf', 'Wolf had to run away, because it was scared of the Human', 'Gazelle ran away, because it was scared of the Lion', 'Lion ran away, because it was scared of the Gazelle', 'Monkey had to run away, because it was scared of the Jaguar', 'Jaguar run away, because it was scared of the Monkey']
['Henry ran away because he was scared of the Wolf. The Wolf is', 'Answer: Lion. Lion was scared of Gazelle because Gazelle was', 'Answer: The Jaguar ran away because it was scared of the Monkey.']


In [ ]:
explanation_tokens =  model.to_tokens(explanation, prepend_bos=True)
print(explanation_tokens)

tensor([[151645,  33975,   1030,    311,   1598,   3123,     11,   1576,    432,
            572,  26115,    315,    279,  25230],
        [151645,  95434,   1030,    311,   1598,   3123,     11,   1576,    432,
            572,  26115,    315,    279,  11097],
        [151645,     38,   1370,   6712,  10613,   3123,     11,   1576,    432,
            572,  26115,    315,    279,  32099],
        [151645,     43,    290,  10613,   3123,     11,   1576,    432,    572,
          26115,    315,    279,  43292,   6712],
        [151645,  96838,   1030,    311,   1598,   3123,     11,   1576,    432,
            572,  26115,    315,    279,  73537],
        [151645,     41,    351,  18731,   1598,   3123,     11,   1576,    432,
            572,  26115,    315,    279,  57837]], device='cuda:0')


In [ ]:
for prompt in prompts:
    str_tokens = model.to_str_tokens(prompt)
    print("Prompt length:", len(str_tokens))
    print("Prompt as tokens:", str_tokens)

for prompt in explanation:
    str_tokens = model.to_str_tokens(prompt)
    print("Explanation length:", len(str_tokens))
    print("Explanation length:", str_tokens)

Prompt length: 33
Prompt as tokens: ['<|im_end|>', 'In', ' the', ' forest', ' two', ' living', ' creatures', ' encountered', ' each', ' other', ',', ' Henry', ' and', ' the', ' Wolf', '.', ' One', ' of', ' them', ' was', ' scared', ',', ' and', ' ran', ' away', '.\n', ' Who', ' did', ' run', ' away', '?', ' Why', '?\n']
Prompt length: 32
Prompt as tokens: ['<|im_end|>', 'In', ' the', ' jungle', ' two', ' living', ' creatures', ' encountered', ' each', ' other', ',', ' Lion', ' and', ' Gaz', 'elle', '.', ' One', ' of', ' them', ' was', ' scared', ' and', ' ran', ' away', '.\n', ' Who', ' did', ' run', ' away', '?', ' Why', '?\n']
Prompt length: 32
Prompt as tokens: ['<|im_end|>', 'In', ' the', ' rain', 'forest', ' two', ' living', ' creatures', ' encountered', ' each', ' other', ',', ' Jaguar', ' and', ' Monkey', '.', ' One', ' of', ' creature', ' was', ' scared', ' and', ' ran', ' away', '.\n', ' Who', ' did', ' run', ' away', '?', ' Why', '?\n']
Explanation length: 14
Explanation leng

In [ ]:
#tokens = model.to_tokens(generation_tokens, prepend_bos=True)
generation_tokens = model.to_tokens(generation,prepend_bos=True)
print(generation_tokens.shape)

# Run the model and cache all activations
generation_logits, cache = model.run_with_cache(generation_tokens)
explanation_logits, cache = model.run_with_cache(explanation_tokens)



torch.Size([3, 15])


In [ ]:
print(generation_logits.shape)
print(explanation_logits.shape)


torch.Size([3, 15, 151936])
torch.Size([6, 14, 151936])


Now, I'll check how Generated Self-Explanations text differs from Explanations

In [ ]:
def calculate_logit_differences(gen_tensor, exp_tensor, per_prompt=False):
    # Extracting final logits for generated text and explanations
    gen_final_logits = gen_tensor[:, -1, :]  # Shape: [3, 151936]
    exp_final_logits = exp_tensor[:, -1, :]  # Shape: [6, 151936]

    # Initializing a tensor to store the individual differences
    individual_differences = []

    # Calculate differences for each generated text with corresponding two explanations
    for i in range(gen_final_logits.shape[0]):
        exp_indices = torch.tensor([2*i, 2*i+1])  # Indices for the corresponding explanations
        for index in exp_indices:
            diff = (gen_final_logits[i] - exp_final_logits[index]).norm()  # Norm of the difference
            individual_differences.append(diff)

    individual_differences = torch.tensor(individual_differences)

    if per_prompt:
        # Calculate mean differences per generated text
        mean_differences = torch.tensor([individual_differences[i:i+2].mean() for i in range(0, len(individual_differences), 2)])
        return individual_differences, mean_differences
    else:
        # Return overall mean of all individual differences
        return individual_differences.mean()

# Calculate differences with per-prompt mean
individual_diffs, mean_diffs = calculate_logit_differences(generation_logits, explanation_logits, per_prompt=True)
print("Logit Differences per Self-Explanation:", individual_diffs.detach().cpu().round(decimals=3))

# Calculate average difference across all pairings
average_diff = calculate_logit_differences(generation_logits, explanation_logits)
print("Average logit difference:", round(average_diff.item(), 3))


Logit Differences per Self-Explanation: tensor([789.9990, 916.5980, 950.1580, 899.3370, 792.7700, 853.8740])
Average logit difference: 867.122


In [ ]:
answer_residual_directions = model.tokens_to_residual_directions(explanation_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = (
    answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
)
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([6, 14, 2048])
Logit difference directions shape: torch.Size([6, 2048])


## Logit Lens

In [ ]:
def residual_stack_to_logit_diff(
    residual_stack: Float[torch.Tensor, "components batch d_model"],
    cache: ActivationCache,
) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(
        residual_stack, layer=-1, pos_slice=-1
    )
    return einsum(
        "... batch d_model, batch d_model -> ...",
        scaled_residual_stack,
        logit_diff_directions,
    ) / len(explanation)

In [ ]:
accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True
)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(
    logit_lens_logit_diffs,
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    hover_name=labels,
    title="Logit Difference From Accumulate Residual Stream",
)

This is actually good news! Since after

## Layer Attribution

In [ ]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

## Head Attribution

In [ ]:
per_head_residual, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True
)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
per_head_logit_diffs = einops.rearrange(
    per_head_logit_diffs,
    "(layer head_index) -> layer head_index",
    layer=model.cfg.n_layers,
    head_index=model.cfg.n_heads,
)
imshow(
    per_head_logit_diffs,
    labels={"x": "Head", "y": "Layer"},
    title="Logit Difference From Each Head",
)

## Attention Analysis

In [ ]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: ActivationCache,
    local_tokens: torch.Tensor,
    title: Optional[str] = "",
    max_width: Optional[int] = 700,
) -> str:
    # If a single head is given, convert to a list
    if isinstance(heads, int):
        heads = [heads]

    # Create the plotting data
    labels: List[str] = []
    patterns: List[Float[torch.Tensor, "dest_pos src_pos"]] = []

    # Assume we have a single batch item
    batch_index = 0

    for head in heads:
        # Set the label
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")

        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # Convert the tokens to strings (for the axis labels)
    str_tokens = model.to_str_tokens(local_tokens)

    # Combine the patterns into a single tensor
    patterns: Float[torch.Tensor, "head_index dest_pos src_pos"] = torch.stack(
        patterns, dim=0
    )

    # Circuitsvis Plot (note we get the code version so we can concatenate with the title)
    plot = attention_heads(
        attention=patterns, tokens=str_tokens, attention_head_names=labels
    ).show_code()

    # Display the title
    title_html = f"<h2>{title}</h2><br/>"

    # Return the visualisation as raw code
    return f"<div style='max-width: {str(max_width)}px;'>{title_html + plot}</div>"

In [ ]:
top_k = 3

top_positive_logit_attr_heads = torch.topk(
    per_head_logit_diffs.flatten(), k=top_k
).indices

positive_html = visualize_attention_patterns(
    top_positive_logit_attr_heads,
    cache,
    generation_tokens[0],
    f"Top {top_k} Positive Logit Attribution Heads",
)

top_negative_logit_attr_heads = torch.topk(
    -per_head_logit_diffs.flatten(), k=top_k
).indices

negative_html = visualize_attention_patterns(
    top_negative_logit_attr_heads,
    cache,
    generation_tokens[0],
    title=f"Top {top_k} Negative Logit Attribution Heads",
)

HTML(positive_html + negative_html)

## Activation Patching